# 4.8 - Grouping Data & Aggregating Variables

### This script contains the following points:

* 01 - Importing Libraries
* 02 - Importing Data
* 03 - Grouping and Aggregating Data
* 04 - Creating a Loyalty Flag
* 05 - Creating a Spender Flag
* 06 - Creating a Frequency Flag
* 07 -  Exporting Data

---

## 01 - Importing Libraries

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

---

## 02 - Importing Data

In [2]:
# Define path

path = r'/Users/juanigalvalisi/01-07-2022 - Instacart Basket Analysis/'

In [3]:
# Import .PKL

df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 - Data', 'Prepared Data', 'orders_products_merged_v2.pkl'))

In [4]:
# Check the output of df_ords_prods_merge

df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_last_order,new_customer,Unnamed: 0,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,outlier_prices,price_range,busiest_day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,24076664,196,1,0,Soda,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,False,22742744,196,1,1,Soda,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders
2,473747,1,3,3,12,21.0,False,4488095,196,1,1,Soda,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders
3,2254736,1,4,4,7,29.0,False,21376074,196,1,1,Soda,77,7,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Average orders
4,431534,1,5,4,15,28.0,False,4089398,196,1,1,Soda,77,7,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Most orders


In [5]:
df_ords_prods_merge.shape

(32404859, 20)

In [6]:
# Ignore maximum of rows

pd.options.display.max_rows = None

---

## 03 - Grouping and Aggregating Data

In [7]:
# Perform Multiple Aggregations

df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
1                15.457838   1  99
2                17.277920   1  99
3                17.170395   1  99
4                17.811403   1  99
5                15.215751   1  99
6                16.439806   1  99
7                17.225802   1  99
8                15.340650   1  99
9                15.895474   1  99
10               20.197148   1  99
11               16.170638   1  99
12               15.887671   1  99
13               16.583536   1  99
14               16.773669   1  99
15               16.165037   1  99
16               17.665606   1  99
17               15.694469   1  99
18               19.310397   1  99
19               17.177343   1  99
20               16.473447   1  99
21               22.902379   1  99

In [8]:
# Compare the full data frame with a one million rows subset

df_subset = df_ords_prods_merge[:1000000]
df_subset.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


> **We now have all of our 21 departments as opposed to the subset where we only had 7. Moreover, all the means are higher with the subset than the entire data frame, ranging from 11,29 to 19,46 and 15,21 to 22,90, respectively.**

---

## 04 - Creating a Loyalty Flag

In [9]:
# Create a loyalty flag for existing customers using the transform() and loc() functions

df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [10]:
# Check the output

df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_last_order,new_customer,Unnamed: 0,product_id,add_to_cart_order,...,product_name,aisle_id,department_id,prices,outlier_prices,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,True,24076664,196,1,...,Soda,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,False,22742744,196,1,...,Soda,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10
2,473747,1,3,3,12,21.0,False,4488095,196,1,...,Soda,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10
3,2254736,1,4,4,7,29.0,False,21376074,196,1,...,Soda,77,7,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Average orders,10
4,431534,1,5,4,15,28.0,False,4089398,196,1,...,Soda,77,7,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Most orders,10


In [11]:
# Establish flag criteria I

df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal Customer'

In [12]:
# Establish flag criteria II

df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular Customer'

In [35]:
# Establish flag criteria III

df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New Customer'

In [14]:
# Check the frequency of the new column, 'loyalty_flag'

df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular Customer    15876776
Loyal Customer      10284093
New Customer         6243990
Name: loyalty_flag, dtype: int64

In [15]:
# Check the final output

df_ords_prods_merge.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_last_order,new_customer,Unnamed: 0,product_id,add_to_cart_order,...,aisle_id,department_id,prices,outlier_prices,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,True,24076664,196,1,...,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer
1,2398795,1,2,3,7,15.0,False,22742744,196,1,...,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer
2,473747,1,3,3,12,21.0,False,4488095,196,1,...,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer
3,2254736,1,4,4,7,29.0,False,21376074,196,1,...,77,7,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Average orders,10,New Customer
4,431534,1,5,4,15,28.0,False,4089398,196,1,...,77,7,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Most orders,10,New Customer
5,3367565,1,6,2,7,19.0,False,31927070,196,1,...,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer
6,550135,1,7,1,9,20.0,False,5212927,196,1,...,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer
7,3108588,1,8,1,14,14.0,False,29474806,196,2,...,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer
8,2295261,1,9,1,16,0.0,False,21760446,196,4,...,77,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer
9,2550362,1,10,4,8,30.0,False,24181266,196,1,...,77,7,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Average orders,10,New Customer


In [16]:
# Check the final output II

df_ords_prods_merge.shape

(32404859, 22)

In [17]:
# Check the basic statistics of the product prices for each new loyalty category

df_ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal Customer    7.774457  1.0  25.0
New Customer      7.802354  1.0  25.0
Regular Customer  7.799319  1.0  25.0

> **As we can see, loyal customers buy the least expensive products, while the new customers acquire, on average, the most expensive products. Regular customers maintain in the middle, but near only a difference of 0,80 from the new customers.**

---

## 05 - Creating a Spender Flag


In [18]:
# Create a new column for average prices for each user

df_ords_prods_merge['mean_user_spending'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [19]:
# Check the output

df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_last_order,new_customer,Unnamed: 0,product_id,add_to_cart_order,...,department_id,prices,outlier_prices,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_user_spending
0,2539329,1,1,2,8,NaN,True,24076664,196,1,...,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367797
1,2398795,1,2,3,7,15.0,False,22742744,196,1,...,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367797
2,473747,1,3,3,12,21.0,False,4488095,196,1,...,7,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer,6.367797
3,2254736,1,4,4,7,29.0,False,21376074,196,1,...,7,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Average orders,10,New Customer,6.367797
4,431534,1,5,4,15,28.0,False,4089398,196,1,...,7,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Most orders,10,New Customer,6.367797


In [20]:
# Establish flag criteria

df_ords_prods_merge.loc[df_ords_prods_merge['mean_user_spending'] < 10, 'spender_flag'] = 'Low Spender'

In [21]:
df_ords_prods_merge.loc[df_ords_prods_merge['mean_user_spending'] >= 10, 'spender_flag'] = 'High Spender'

In [22]:
# Check the frequency of the new column, 'spender_flag'

df_ords_prods_merge['spender_flag'].value_counts(dropna = False)

Low Spender     32284898
High Spender      119961
Name: spender_flag, dtype: int64

In [23]:
# Check the final output

df_ords_prods_merge.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_last_order,new_customer,Unnamed: 0,product_id,add_to_cart_order,...,prices,outlier_prices,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_user_spending,spender_flag
0,2539329,1,1,2,8,NaN,True,24076664,196,1,...,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367797,Low Spender
1,2398795,1,2,3,7,15.0,False,22742744,196,1,...,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367797,Low Spender
2,473747,1,3,3,12,21.0,False,4488095,196,1,...,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer,6.367797,Low Spender
3,2254736,1,4,4,7,29.0,False,21376074,196,1,...,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Average orders,10,New Customer,6.367797,Low Spender
4,431534,1,5,4,15,28.0,False,4089398,196,1,...,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Most orders,10,New Customer,6.367797,Low Spender
5,3367565,1,6,2,7,19.0,False,31927070,196,1,...,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367797,Low Spender
6,550135,1,7,1,9,20.0,False,5212927,196,1,...,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer,6.367797,Low Spender
7,3108588,1,8,1,14,14.0,False,29474806,196,2,...,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer,6.367797,Low Spender
8,2295261,1,9,1,16,0.0,False,21760446,196,4,...,9.0,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer,6.367797,Low Spender
9,2550362,1,10,4,8,30.0,False,24181266,196,1,...,9.0,Not Outlier,Mid-range product,Least busy,Least busy,Average orders,10,New Customer,6.367797,Low Spender


In [24]:
df_ords_prods_merge.shape

(32404859, 24)

> **In general terms, loyal customers buy the least expensive products, while the new customers adquire, on average, the most expensive products. Regular customers maintain in the middle, but near from only a difference of 0,80 from the new customers.**

---

## 06 - Creating a Frequency Flag

In [25]:
# Create an order frequency flag that marks the regularity of a user’s ordering
# behavior according to the median in the “days_last_order” column

df_ords_prods_merge['order_frequency_median'] = df_ords_prods_merge.groupby(['user_id'])['days_last_order'].transform(np.median)

In [26]:
# Check the output

df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_last_order,new_customer,Unnamed: 0,product_id,add_to_cart_order,...,outlier_prices,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_user_spending,spender_flag,order_frequency_median
0,2539329,1,1,2,8,NaN,True,24076664,196,1,...,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367797,Low Spender,20.5
1,2398795,1,2,3,7,15.0,False,22742744,196,1,...,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367797,Low Spender,20.5
2,473747,1,3,3,12,21.0,False,4488095,196,1,...,Not Outlier,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer,6.367797,Low Spender,20.5
3,2254736,1,4,4,7,29.0,False,21376074,196,1,...,Not Outlier,Mid-range product,Least busy,Least busy,Average orders,10,New Customer,6.367797,Low Spender,20.5
4,431534,1,5,4,15,28.0,False,4089398,196,1,...,Not Outlier,Mid-range product,Least busy,Least busy,Most orders,10,New Customer,6.367797,Low Spender,20.5


In [27]:
df_ords_prods_merge.shape

(32404859, 25)

In [28]:
# Establish flag criteria

df_ords_prods_merge.loc[df_ords_prods_merge['days_last_order'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [29]:
df_ords_prods_merge.loc[(df_ords_prods_merge['days_last_order']  > 10) & (df_ords_prods_merge['days_last_order'] <= 20), 'order_frequency_flag'] = 'Regular customer'

In [30]:
df_ords_prods_merge.loc[df_ords_prods_merge['days_last_order'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [31]:
# Check the frequency of the new column, 'order_frequency_flag'

df_ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

Frequent customer        19042466
Regular customer          6055803
Non-frequent customer     5230494
NaN                       2076096
Name: order_frequency_flag, dtype: int64

In [32]:
# Check the output

df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_last_order,new_customer,Unnamed: 0,product_id,add_to_cart_order,...,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_user_spending,spender_flag,order_frequency_median,order_frequency_flag
0,2539329,1,1,2,8,NaN,True,24076664,196,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367797,Low Spender,20.5,NaN
1,2398795,1,2,3,7,15.0,False,22742744,196,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New Customer,6.367797,Low Spender,20.5,Regular customer
2,473747,1,3,3,12,21.0,False,4488095,196,1,...,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New Customer,6.367797,Low Spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,False,21376074,196,1,...,Mid-range product,Least busy,Least busy,Average orders,10,New Customer,6.367797,Low Spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,False,4089398,196,1,...,Mid-range product,Least busy,Least busy,Most orders,10,New Customer,6.367797,Low Spender,20.5,Non-frequent customer


In [33]:
df_ords_prods_merge.shape

(32404859, 26)

> **Frequent customers made almost 60% of the orders. At the same time, the NaN values display the number of purchases made by customers who did buy something only once in their history as customers in Instacart.**

---

## 07- Exporting Data

In [34]:
# Export .pkl data frame

df_ords_prods_merge.to_pickle(os.path.join(path, '02 - Data', 'Prepared Data', 'orders_products_merged_v3.pkl'))